# Python で気軽に化学・化学工学
# 第 7 章 データセットを類似するサンプルごとにグループ化する

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 数値データの特徴量のみを x に (あやめのデータでは 0 列目が Species でカテゴリーの特徴量であるため、それ以外の特徴量を取り出しています)

In [ ]:
x # 念のため確認します

In [ ]:
iris_types = dataset.iloc[:, 0] # あやめの種類を iris_type としておきます

In [ ]:
iris_types # 念のため確認します

## 階層的クラスタリングの実行

ここでは樹形図の結果を確認しやすくするため、150 のあやめから setosa, versicolor, virginica それぞれ 5 サンプルずつ選択して、合計 15 サンプルのみで階層的クラスタリングをします。最初のデモンストレーションのためにサンプルを選択しているだけであり、特に意味はありません。基本的には、すべてのサンプルで階層的クラスタリングをします。再度、上のセルでデータセットを読み込んで x, iris_types を準備してから、下のセルを実行せずに、さらに次のセルから実行すれば、すべてのサンプルでクラスタリングできます。

In [ ]:
selected_sample_numbers = [0, 1, 2, 3, 4, 59, 60, 64, 79, 81, 102, 105, 107, 109, 117]
x = x.iloc[selected_sample_numbers, :]
iris_types = iris_types.iloc[selected_sample_numbers]

In [ ]:
x # 念のため確認

In [ ]:
iris_types # 念のため確認

特徴量の標準化

In [ ]:
autoscaled_x = (x - x.mean(axis=0)) / x.std(axis=0, ddof=1)

階層的クラスタリング

### SciPy
- Python における統計・最適化・信号処理などの行うためのライブラリ
- クラスタリング・スペクトル解析などにおける様々な手法や距離計算などに関するプログラムを利用できる
- オフィシャルウェブサイトは [こちら](https://www.scipy.org/)

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster # SciPy の中の階層的クラスタリングを実行したり樹形図を作成したりするためのライブラリを取り込む

In [ ]:
clustering_results = linkage(autoscaled_x, metric='euclidean', method='ward')

`metric`, `method` を下のように変えることで、それぞれ色々な距離、手法で階層的クラスタリングを実行できます。最初は、ユークリッド距離・ウォード法となっています。

`metric` の種類
- euclidean : ユークリッド距離
- cityblock : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html) をご覧ください 

`method` の種類
- single : 最近隣法
- complete : 最遠隣法
- weighted : 重心法
- average : 平均距離法
- ward : ウォード法

など。その他の手法については [こちら](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) をご覧ください 

デンドログラムの作成

In [ ]:
import matplotlib.pyplot as plt # 描画のためインポート

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
dendrogram(clustering_results, labels=x.index, color_threshold=0, orientation='right') # デンドログラムの作成。labels=x.index でサンプル名を入れています
plt.xlabel('distance') # 横軸の名前
plt.show()

あやめの種類を樹形図で確認

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
dendrogram(clustering_results, labels=iris_types, color_threshold=0, orientation='right') # デンドログラムの作成。labels=iris_types であやめの種類を入れています
plt.xlabel('distance') # 横軸の名前
plt.show()

主成分分析 (Principal Component Analysis, PCA) による階層的クラスタリングの結果の可視化

In [ ]:
from sklearn.decomposition import PCA # scikit-learn の中の PCA を実行するためのライブラリを取り込む

In [ ]:
pca = PCA() # PCA を行ったり PCA の結果を格納したりするための変数を、pca として宣言

In [ ]:
pca.fit(autoscaled_x) # 変数の標準化後のデータを用いて、PCA を実行

In [ ]:
pca.transform(autoscaled_x) # 主成分スコア T の計算。array 型で得られます

In [ ]:
score = pd.DataFrame(pca.transform(autoscaled_x)) # データ型を、使い慣れた pandas の DataFrame 型に変換

In [ ]:
score # 念のため確認

In [ ]:
score.index = x.index # スコアのサンプル名を、元のデータのサンプル名に

In [ ]:
pc_names = ['PC1', 'PC2', 'PC3', 'PC4'] # 主成分の名前 PC1, PC2, ... 。
# 主成分の数に合わせて作成する必要があります。for 文を学べば、もっと効率的に作成可能でして、連載のどこかでやる予定です

In [ ]:
score.columns = pc_names # スコアの列の名前を、PC1, PC2, ... に

In [ ]:
score # 念のため確認

寄与率、累積寄与率

In [ ]:
pca.explained_variance_ratio_ # 寄与率。array 型で得られます

In [ ]:
contribution_ratios = pd.DataFrame(pca.explained_variance_ratio_) # DataFrame 型に変換

In [ ]:
contribution_ratios.columns = ['contribution_ratio'] # 列名を変更

In [ ]:
contribution_ratios # 確認

In [ ]:
cumulative_contribution_ratios = contribution_ratios.cumsum() # cumsum() で寄与率の累積和を計算

In [ ]:
cumulative_contribution_ratios.columns = ['cumulative_contribution_ratio'] # 列名を変更

In [ ]:
cumulative_contribution_ratios # 確認

In [ ]:
x_axis = range(1, contribution_ratios.shape[0] + 1) # 1 から成分数までの整数が x 軸の値

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.bar(x_axis, contribution_ratios.iloc[:, 0], align='center') # 寄与率の棒グラフ
plt.plot(x_axis, cumulative_contribution_ratios.iloc[:, 0], 'r.-') # 累積寄与率の線を入れたプロット図
plt.xlabel('Number of principal components') # 横軸の名前
plt.ylabel('Contribution ratio(blue),\nCumulative contribution ratio(red)') # 縦軸の名前。\n で改行しています
plt.show()

データセット・クラスタリング結果の可視化。２つの主成分ごとのプロットにおいて、サンプル点の色をクラスターごとに変えることで、クラスターの様子を可視化します。最初にクラスター数を決める必要があります

In [ ]:
number_of_clusters = 3 # クラスターの数

In [ ]:
cluster_numbers = fcluster(clustering_results, number_of_clusters, criterion='maxclust') # クラスターの数で分割し、クラスター番号を出力

In [ ]:
cluster_numbers = pd.DataFrame(cluster_numbers) # DataFrame 型に変換

In [ ]:
cluster_numbers.columns = ['cluster_numbers'] # 列名を変更

In [ ]:
cluster_numbers.index = x.index # クラスター番号のサンプル名を、元のデータのサンプル名に

In [ ]:
cluster_numbers # 念のため確認

In [ ]:
cluster_numbers.to_csv('cluster_numbers.csv') # クラスター番号を csv ファイルに保存

cluster_numbers.csv を Excel 等で開いて中身を確認しましょう。

主成分スコアをプロットしてデータセットの確認をします。今回は、component_number_1 番目の主成分と component_number_2 番目の主成分との間の散布図を描画するプログラムとします。

In [ ]:
component_number_1 = 0
component_number_2 = 1

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2],  c=cluster_numbers.iloc[:, 0], cmap=plt.get_cmap('jet')) # 散布図の作成。クラスター番号ごとにプロットの色を変えています
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

主成分間のプロットにおいて、クラスターごとにサンプルの色を変えて描画することで、クラスタリングの結果を可視化できました。ただし、クラスタリングは元の変数で行っており、可視化は PCA 後の主成分で行っており、元の変数と主成分とで異なるため、散布図において近いサンプルが必ずしも同じクラスターとは限りません。

クラスターの数 number_of_clusters や主成分の番号 component_number_1, component_number_2 を変えて、クラスターの様子を確認してみましょう。

ここまで、最初に選択した 15 サンプルのみで行ってきましたが、すべての 150 サンプルを用いて、クラスタリングや PCA による可視化を行ってみましょう。再度データセットを読み込むため、最初からセルを実行して、15 サンプルを選択したセル (`x = x.iloc[selected_sample_numbers, :]` を含む三行のコードが書いてあるセル) を実行せずに飛ばして、その後は普通に実行することでできます。

### 【参考】
主成分間のプロットにおいて、あやめの種類ごとにサンプルの色を変えて描画してみます。クラスタリングの結果と比較してみましょう。

In [ ]:
pd.factorize(iris_types)[0] # あやめの種類を、0, 1, 2 に変換。この数値に基づいて、点の色を変えて散布図を描画します

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(score.iloc[:, component_number_1], score.iloc[:, component_number_2],  c=pd.factorize(iris_types)[0], cmap=plt.get_cmap('jet')) # 散布図の作成。あやめの種類ごとにプロットの色を変えています
plt.xlabel(score.columns[component_number_1]) # 横軸の名前。ここでは、component_number_1 番目の列の名前にしています
plt.ylabel(score.columns[component_number_2]) # 縦軸の名前。ここでは、component_number_2 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

### 練習問題

データセット `virtual_equipment.csv` を読み込み、2 つの温度 (`T1`, `T2`) のみを特徴量として用いて (`state` は用いない) 階層的クラスタリングを行い、散布図により結果を可視化しましょう。クラスター数を変えて結果を比較してみましょう。

### 仮想的な装置のデータセット (virtual_equipment.csv)
仮想的な装置において、時刻ごとの 2 つの温度の測定値 (`T1`, `T2`) とそのときの装置の状態 (`state`) が入ったデータセット。`state` の種類としては、normal (正常)、abnormal_1 (異常1)、abnormal_2 (異常2)、abnormal_3 (異常3) があります。

### 練習問題 コードの例

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('virtual_equipment.csv', index_col=0) # 仮想的な装置のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 0:2] # 温度の特徴量のみを x に

In [ ]:
x # 念のため確認します

In [ ]:
states = dataset.iloc[:, 2] # 装置の状態を states としておきます

In [ ]:
states # 念のため確認

特徴量の標準化

In [ ]:
autoscaled_x = (x - x.mean(axis=0)) / x.std(axis=0, ddof=1)

階層的クラスタリング

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster # SciPy の中の階層的クラスタリングを実行したり樹形図を作成したりするためのライブラリを取り込む

In [ ]:
clustering_results = linkage(autoscaled_x, metric='euclidean', method='ward')

データセット・クラスタリング結果の可視化。２つの特徴量の散布図において、サンプル点の色をクラスターごとに変えることで、クラスターの様子を可視化します。最初にクラスター数を決める必要があります

In [ ]:
number_of_clusters = 5 # クラスターの数

In [ ]:
cluster_numbers = fcluster(clustering_results, number_of_clusters, criterion='maxclust') # クラスターの数で分割し、クラスター番号を出力

In [ ]:
cluster_numbers = pd.DataFrame(cluster_numbers) # DataFrame 型に変換

In [ ]:
cluster_numbers.columns = ['cluster_numbers'] # 列名を変更

In [ ]:
cluster_numbers.index = x.index # クラスター番号のサンプル名を、元のデータのサンプル名に

In [ ]:
cluster_numbers # 念のため確認

In [ ]:
cluster_numbers.to_csv('cluster_numbers.csv') # クラスター番号を csv ファイルに保存

散布図の描画

In [ ]:
import matplotlib.pyplot as plt # 描画のためインポート

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(x.iloc[:, 0], x.iloc[:, 1],  c=cluster_numbers.iloc[:, 0], cmap=plt.get_cmap('jet')) # 散布図の作成。クラスター番号ごとにプロットの色を変えています
plt.xlabel(x.columns[0]) # 横軸の名前。ここでは、0 番目の列の名前にしています
plt.ylabel(x.columns[1]) # 縦軸の名前。ここでは、1 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します

2 つの特徴量間の散布図において、装置の状態ごとにサンプルの色を変えて描画してみます。クラスタリングの結果と比較してみましょう。

In [ ]:
pd.factorize(states)[0] # 装置の状態を、0, 1, ... に変換。この数値に基づいて、点の色を変えて散布図を描画します

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(x.iloc[:, 0], x.iloc[:, 1],  c=pd.factorize(states)[0], cmap=plt.get_cmap('jet')) # 散布図の作成。クラスター番号ごとにプロットの色を変えています
plt.xlabel(x.columns[0]) # 横軸の名前。ここでは、0 番目の列の名前にしています
plt.ylabel(x.columns[1]) # 縦軸の名前。ここでは、1 番目の列の名前にしています
plt.show() # 以上の設定において、グラフを描画します